In [187]:
import pandas as pd
import chordparser
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

---------------

### Definitions

In [188]:
cp = chordparser.Parser()
CE = chordparser.ChordEditor()
CRC = chordparser.ChordRomanConverter()

In [189]:
sharps_to_flats = {
    'C#': 'Db',
    'D#': 'Eb',
    'F#': 'Gb',
    'G#': 'Ab',
    'A#': 'Bb',
}

In [190]:
lower_base = {
    'C': 'B',
    'B': 'Bb',
    'Bb': 'A',
    'A': 'Ab',
    'Ab': 'G',
    'G': 'Gb',
    'Gb': 'F',
    'F': 'E',
    'E': 'Eb',
    'Eb': 'D',
    'D': 'Db',
    'Db': 'C'
}

In [191]:
number_of_songs = {'Country': 0,
                   'Electronic': 0,
                   'Pop': 0,
                   'Rock': 0}

In [192]:
# {'Country': {'IV': 30, 'V': 2, ...}, 'Pop': {...}, ...}
nodelists_sum = {'Country': {},
                 'Electronic': {},
                 'Pop': {},
                 'Rock': {}}

In [193]:
dataframes = {'Country': pd.DataFrame(columns=['from_roman', 'to_roman']),
              'Electronic': pd.DataFrame(columns=['from_roman', 'to_roman']),
              'Pop': pd.DataFrame(columns=['from_roman', 'to_roman']),
              'Rock': pd.DataFrame(columns=['from_roman', 'to_roman'])}

In [194]:
nodelists = {}

----------------------

### Single song processing

In [195]:
def get_roman_dataframe(file):
    # dataframe
    df = pd.read_csv(file, delimiter='\t', skiprows=2, header=None)
    df.columns=['from', 'to']

    # signature
    with open(file, "r") as fin:
        signature = fin.readlines()[:2]
        
    base = re.split(':|\\n', signature[0])[1]
    capo = re.split(':|\\n', signature[1])[1]
    
    if capo == 'None':
        capo = 0
    else:
        capo = int(capo)

    # key and scale
    if('m' in base):
        mode = 'minor'
        base = base.replace('m', '')
    else:
        mode = 'major'
        
    if base in sharps_to_flats.keys() and base != df['from'][0].replace('m', ''):
        base = sharps_to_flats[base]  

    for i in range(capo):
        base = lower_base[base]
    
    key = cp.create_key(base, mode)

    df['from'] = df['from'].apply(lambda x: x.replace(')', '').replace('(', '').replace('-5', 'b5').replace('+5', '#5').replace('-', ''))#.replace('5', '').replace('-', ''))
    df['to'] = df['to'].apply(lambda x: x.replace(')', '').replace('(', '').replace('-5', 'b5').replace('+5', '#5').replace('-', ''))#.replace('5', '').replace('-', ''))

    # convert to roman
    if base in sharps_to_flats.keys():
        base = sharps_to_flats[base]  
        
    df['from_roman'] = df['from'].apply(lambda x: str(CRC.to_roman(CE.create_chord(x),key)))
    df['to_roman'] = df['to'].apply(lambda x: str(CRC.to_roman(CE.create_chord(x),key)))
    
    df = df.drop(columns=['from','to'])

    return df

In [196]:
def add_to_nodelist(df, genre):    
    all_chords = pd.unique(df.values.ravel('K'))
    
    dataframes[genre] = dataframes[genre].append(df)
        
    for chord in all_chords:
        value_count = 0
        
        if chord in df.from_roman.values:
            value_count = df.from_roman.value_counts()[chord]

        # include last chord
        if (df.to_roman.iloc[-1]) == chord:
            value_count += 1

        # add appearances to nodelist
        if chord not in nodelists_sum[genre].keys():
            nodelists_sum[genre][chord] = 0
        nodelists_sum[genre][chord] += value_count

    return

In [197]:
def process_song(file):

    df = get_roman_dataframe(file)
    
    genre = file.split('/')[1]
    add_to_nodelist(df, genre)

    number_of_songs[genre] += 1

### Genre processing

In [198]:
def create_nodelist(genre):
    df = pd.DataFrame([nodelists_sum[genre]])
    df = df.T.rename(columns={0: 'sum_appearance'})
    
    df['percent'] = df['sum_appearance'] / df['sum_appearance'].sum()
    df['average_appearance'] = df['sum_appearance'] / number_of_songs[genre]

    df = df.reset_index()
    df = df.rename(columns={'index': 'Id'})
    
    return df

In [202]:
def get_percentages(df, nodelist):

    df = df.groupby(df.columns.tolist(),as_index=False).size()
    df['percent'] = df['size'] / df['size'].sum()
    
    df = df.rename(columns={'size': 'sum_appearance', 'from_roman': 'Source', 'to_roman': 'Target'})

    log_ratio = []
    for i, row in df.iterrows():
        source_percent = nodelist.loc[nodelist['Id'] == row.Source, 'percent'].iloc[0]
        target_percent = nodelist.loc[nodelist['Id'] == row.Target, 'percent'].iloc[0]
        log_ratio.append(np.log(row.percent / (source_percent * target_percent)))

    df['log_ratio'] = log_ratio

    return df

--------------

## Main

### Process every song

In [200]:
directory = 'data/'

for subdir, dirs, files in os.walk(directory):
    for i, filename in enumerate(os.listdir(subdir)):
        f = os.path.join(subdir, filename)
        if os.path.isfile(f):
            df = process_song(f)

### Process genres

In [203]:
for genre, df in dataframes.items():
    nodelists[genre] = create_nodelist(genre)
    dataframes[genre] = get_percentages(df, nodelists[genre])

### Save nodelists and edgelists

In [204]:
for genre, df in dataframes.items():
    df.to_csv('edge_lists/edges_' + genre.lower() + '.csv', index=False)
    nodelists[genre].to_csv('node_lists/nodes_' + genre.lower() + '.csv', index=False)

In [205]:
dataframes['Country']

,Source,Target,sum_appearance,percent,log_ratio
0,I,I,142,0.054657,-0.361174
1,I,I6,7,0.002694,0.832244
2,I,I7,6,0.002309,0.996547
3,I,II,1,0.000385,-3.685584
4,I,III,1,0.000385,-0.102065
...,...,...,...,...,...
140,♭VII,vi,2,0.000770,-0.520131
141,♭VII,♭VII,6,0.002309,0.768524
142,♭vi°,vi,2,0.000770,3.241069
143,♯iv,V,6,0.002309,1.794150


In [ ]:
nodelists['Country']

,id,sum_appearance,percent,average_appearance
0,IV64,62,0.023592,2.066667
1,I,736,0.280061,24.533333
2,vi,104,0.039574,3.466667
3,V,442,0.168189,14.733333
4,ii,116,0.044140,3.866667
5,IV,423,0.160959,14.100000
6,♭VII,86,0.032725,2.866667
7,I6,11,0.004186,0.366667
8,V7,15,0.005708,0.500000
9,I7,8,0.003044,0.266667


- heatmap of matrix
- graphs for genres
- size of edges (how often does it appear/how probable)
- more pruning (threshold for edges/nodes)